# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html
- Step by step vidio tutorial
    - https://www.youtube.com/watch?v=ab1mbj0acDo

## 1. role setting (adding trust relationship)

### 1.1. role check

In [ ]:
from sagemaker import get_execution_role

In [ ]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../imgs/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../imgs/inline-policy.jpg)

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

### 1.4. Trust relationship
![nn](../imgs/trust-relationship.jpg)

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

In [ ]:
install_needed = True

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain #==0.0.50
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy #==2.0.1
    !{sys.executable} -m pip install -U pypdf
    !{sys.executable} -m pip install -U spacy
    !{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython
    !{sys.executable} -m pip install -U ipywidgets
    !{sys.executable} -m pip install -U llmsherpa
    !{sys.executable} -m pip install -U anthropic
    !{sys.executable} -m pip install -U faiss-cpu


    IPython.Application.instance().kernel.do_shutdown(True)

## 4. Check setting
모델 리스트 <BR>
![nn](../imgs/check.png)


In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [12]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [13]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

langchain version check: 0.1.4
boto3 version check: 1.34.29
botocore version check: 1.34.29
awscli version check: 1.32.29


**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [14]:
bedrock_region = "us-east-1"

In [15]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

In [16]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 5. generation

In [17]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Titan

## Titan text모델은 현재 preview 서비스입니다. GA 이후 업데이트 하도록 할께요! (현재 에러발생)

In [ ]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-G1"),
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [ ]:
print (colored(llm(prompt), "green"))

### Claude v2.1

In [18]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:"],
        "temperature":0,
        "top_p":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [19]:
prompt = "\n\nHuman: 세이지메이커의 장점을 100단어로 알려주세요 \n\nAssistant:"

In [ ]:
response = llm(prompt)

* Long text to examine 200k-input (optional)

In [ ]:
text_file = "../utils/sample-text/long-text-70k-tokens.txt"
with open(text_file, "r") as file:
    text = file.read()
    
prompt = "".join(["\n\nHuman: 아래 글을 요약해줘\n\n", text, "\n\nAssistant:"])
    
print (f"# characters in prompt: {len(prompt)}")

In [ ]:
# response = llm(prompt)

In [23]:
prompt = '''
\n\nHuman: You are a writing robot. I'm going to give you a context. Read the context carefully, because I’m going to ask you write a post based on the given context.

Here is the context: <context>
해결책을 살펴보기 전에 먼저 문제에 대해 이야기해 보겠습니다. RAG를 사용하면 수만 개에서 최대 수백억 개에 이르는 수많은 텍스트 문서에서 의미론적 검색을 수행하게 됩니다.

대규모 검색 시간을 단축하기 위해 일반적으로 벡터 검색을 사용합니다. 즉, 텍스트를 벡터로 변환하여 벡터 공간에 모두 배치하고 코사인 유사도와 같은 유사성 메트릭을 사용하여 쿼리 벡터와의 근접성을 비교하는 것입니다.

벡터 검색이 작동하려면 벡터가 필요합니다. 이러한 벡터는 기본적으로 일부 텍스트 뒤에 있는 '의미'를 (일반적으로) 768 또는 1536 차원 벡터로 압축한 것입니다. 이 정보를 단일 벡터로 압축하기 때문에 약간의 정보 손실이 발생합니다.

이러한 정보 손실로 인해 상위 3개(예: 벡터 검색 문서)의 관련 정보가 누락되는 경우가 종종 있습니다. 안타깝게도 검색 결과 상위_k 컷오프 아래에 있는 관련 정보가 반환될 수 있습니다.

더 낮은 위치에 있는 관련 정보가 LLM이 더 나은 답변을 작성하는 데 도움이 된다면 어떻게 해야 할까요? 가장 쉬운 방법은 반환하는 문서 수를 늘려서(top_k 증가) 모든 문서를 LLM에 전달하는 것입니다.

여기서 측정할 메트릭은 "얼마나 많은 관련 문서를 검색하는가"를 의미하는 리콜입니다. 리콜은 검색된 문서의 총 개수를 고려하지 않으므로 메트릭을 해킹하여 모든 문서를 반환함으로써 완벽한 리콜을 얻을 수 있습니다.

안타깝게도 모든 것을 반환할 수는 없습니다. LLM에는 전달할 수 있는 텍스트의 양에 제한이 있으며, 이를 컨텍스트 창이라고 합니다. 일부 LLM은 컨텍스트 창이 10만 토큰[1]인 Anthropic의 Claude와 같이 거대한 컨텍스트 창을 가지고 있습니다. 이 정도면 수십 페이지의 텍스트를 담을 수 있는데, 그렇다면 많은 문서를 반환하고(전부는 아니지만) 컨텍스트 창을 '채우면' 리콜을 개선할 수 있을까요?

다시 말하지만, 안 됩니다. 컨텍스트 스터핑은 LLM의 리콜 성능을 저하시키기 때문에 사용할 수 없습니다. 이것은 지금까지 논의한 검색 리콜과는 다른 LLM 리콜이라는 점에 유의하세요.

LLM 리콜은 컨텍스트 창에 배치된 텍스트에서 정보를 찾는 LLM의 능력을 말합니다. 연구에 따르면 컨텍스트 창에 더 많은 토큰을 넣을수록 LLM 리콜이 저하되는 것으로 나타났습니다[2]. 또한 컨텍스트 창에 토큰을 채우면 LLM이 지시를 따를 가능성도 낮아지므로 컨텍스트 채우기는 좋지 않습니다.

벡터 DB가 반환하는 문서 수를 늘려 검색 회상률을 높일 수는 있지만, LLM 회상률을 손상시키지 않고는 이를 LLM에 전달할 수 없습니다.

이 문제에 대한 해결책은 많은 문서를 검색하여 검색 회상률을 극대화한 다음, LLM에 전달되는 문서 수를 최소화하여 LLM 회상률을 극대화하는 것입니다. 이를 위해 검색된 문서의 순서를 바꾸고 LLM에 가장 관련성이 높은 문서만 유지하는데, 이를 위해 재순위를 사용합니다.

</context>

Only using the context as above, write a post with the rules as below:
                - Don't insert XML tag such as <context> and </context> when answering.
                - Write as much as you can
                - Be courteous and polite
                - Only answer the question if you can find the answer in the context with certainty.
                - 한글로 해주세요

\n\nAssistant:

'''
response = llm(prompt)

 <답변>

문제 상황을 잘 이해했습니다. 

벡터 검색을 통해 수많은 문서에서 검색을 수행하면 검색 회상률은 높아지지만, Language Model에 전달할 수 있는 문서 수에는 제한이 있습니다. 

Language Model에 전달되는 문서 수를 늘리면 Language Model의 회상률이 떨어지는 문제가 발생합니다.

이 문제를 해결하기 위해, 벡터 검색을 통해 수많은 문서를 검색하여 검색 회상률을 극대화한 후, Language Model에 전달되는 문서 수를 최소화하여 Language Model 회상률을 극대화하는 것이 좋습니다. 

구체적으로 재순위 알고리즘을 사용하여 검색된 문서의 순서를 조정하고, Language Model에 가장 관련성이 높은 문서만 유지하는 것이 효과적인 해결책이 될 것 같습니다.

</답변>

In [26]:
prompt = '''
\n\nHuman: You are a writing robot. I'm going to give you a context. Read the context carefully, because I’m going to ask you rephrase the given context differently.

Rephrase the post differently
Here is the context: <context>
해결책을 살펴보기 전에 먼저 문제에 대해 이야기해 보겠습니다. RAG를 사용하면 수만 개에서 최대 수백억 개에 이르는 수많은 텍스트 문서에서 의미론적 검색을 수행하게 됩니다.

대규모 검색 시간을 단축하기 위해 일반적으로 벡터 검색을 사용합니다. 즉, 텍스트를 벡터로 변환하여 벡터 공간에 모두 배치하고 코사인 유사도와 같은 유사성 메트릭을 사용하여 쿼리 벡터와의 근접성을 비교하는 것입니다.

벡터 검색이 작동하려면 벡터가 필요합니다. 이러한 벡터는 기본적으로 일부 텍스트 뒤에 있는 '의미'를 (일반적으로) 768 또는 1536 차원 벡터로 압축한 것입니다. 이 정보를 단일 벡터로 압축하기 때문에 약간의 정보 손실이 발생합니다.

이러한 정보 손실로 인해 상위 3개(예: 벡터 검색 문서)의 관련 정보가 누락되는 경우가 종종 있습니다. 안타깝게도 검색 결과 상위_k 컷오프 아래에 있는 관련 정보가 반환될 수 있습니다.

더 낮은 위치에 있는 관련 정보가 LLM이 더 나은 답변을 작성하는 데 도움이 된다면 어떻게 해야 할까요? 가장 쉬운 방법은 반환하는 문서 수를 늘려서(top_k 증가) 모든 문서를 LLM에 전달하는 것입니다.

여기서 측정할 메트릭은 "얼마나 많은 관련 문서를 검색하는가"를 의미하는 리콜입니다. 리콜은 검색된 문서의 총 개수를 고려하지 않으므로 메트릭을 해킹하여 모든 문서를 반환함으로써 완벽한 리콜을 얻을 수 있습니다.

안타깝게도 모든 것을 반환할 수는 없습니다. LLM에는 전달할 수 있는 텍스트의 양에 제한이 있으며, 이를 컨텍스트 창이라고 합니다. 일부 LLM은 컨텍스트 창이 10만 토큰[1]인 Anthropic의 Claude와 같이 거대한 컨텍스트 창을 가지고 있습니다. 이 정도면 수십 페이지의 텍스트를 담을 수 있는데, 그렇다면 많은 문서를 반환하고(전부는 아니지만) 컨텍스트 창을 '채우면' 리콜을 개선할 수 있을까요?

다시 말하지만, 안 됩니다. 컨텍스트 스터핑은 LLM의 리콜 성능을 저하시키기 때문에 사용할 수 없습니다. 이것은 지금까지 논의한 검색 리콜과는 다른 LLM 리콜이라는 점에 유의하세요.

LLM 리콜은 컨텍스트 창에 배치된 텍스트에서 정보를 찾는 LLM의 능력을 말합니다. 연구에 따르면 컨텍스트 창에 더 많은 토큰을 넣을수록 LLM 리콜이 저하되는 것으로 나타났습니다[2]. 또한 컨텍스트 창에 토큰을 채우면 LLM이 지시를 따를 가능성도 낮아지므로 컨텍스트 채우기는 좋지 않습니다.

벡터 DB가 반환하는 문서 수를 늘려 검색 회상률을 높일 수는 있지만, LLM 회상률을 손상시키지 않고는 이를 LLM에 전달할 수 없습니다.

이 문제에 대한 해결책은 많은 문서를 검색하여 검색 회상률을 극대화한 다음, LLM에 전달되는 문서 수를 최소화하여 LLM 회상률을 극대화하는 것입니다. 이를 위해 검색된 문서의 순서를 바꾸고 LLM에 가장 관련성이 높은 문서만 유지하는데, 이를 위해 재순위를 사용합니다.

</context>

Only using the context as above, write a post with the rules as below:
                - Don't insert XML tag such as <context> and </context> when answering.
                - Write as much as you can
                - Be courteous and polite
                - 한글로 해주세요

\n\nAssistant:

'''
response = llm(prompt)

 문제 해결을 위해서는 먼저 문제의 본질을 이해하는 것이 중요합니다. 

이 경우, 대규모 텍스트 데이터베이스에서 의미 있는 정보를 검색하고자 하는 데 어려움이 있습니다. 벡터 기반 검색은 빠르지만 정보 손실로 인해 관련성 있는 결과를 놓치는 경우가 발생합니다. 

한편으로는 많은 문서를 검색하여 검색 결과의 완전성을 높이고 싶지만, 문서가 너무 많으면 언어모델의 이해 능력이 떨어지고 정보를 제대로 활용하지 못하는 문제가 발생합니다.

이에 대한 해결책은 벡터 검색을 통해 가능한 많은 문서를 검색한 후, 문서의 관련성을 재평가하여 언어모델에 제공할 최적의 문서 집합을 구성하는 것입니다. 

즉, 검색 완전성과 언어모델 활용 효율성을 동시에 극대화할 수 있는 최적의 균형점을 찾는 것이 핵심입니다. 이를 위해 문서의 재순위 알고리즘이 도움이 될 수 있습니다.

포괄적이고 체계적인 접근이 필요한 문제인 만큼, 다각적인 시도를 통해 최적의 해결책을 찾아가야 할 것 같습니다.